In [124]:
import pandas as pd
import numpy as np
from pathlib import Path


In [125]:

# Open resources csvs file
crime_data = Path("../Resources/crime_location.csv")
# Open resources csvs file station data
station_data = Path("../Resources/station_name.csv")


In [126]:
# Read the crime data 
crime = pd.read_csv(crime_data)
crime.head()


,location
0,Millbrae BART Station
1,Balboa Park BART Station
2,Embarcadero BART Station
3,Milpitas BART Station
4,Richmond BART Station


In [127]:
# Remove BART and Station expressions from the crime data
new_crime_data = crime['location'].str.replace(' BART', '')
new_crime_data = new_crime_data.str.replace(' Station', '')

pd.DataFrame(new_crime_data).head()

,location
0,Millbrae
1,Balboa Park
2,Embarcadero
3,Milpitas
4,Richmond


In [128]:
# read the station data
new_station_data = pd.read_csv(station_data)
new_station_data.head()


,name
0,12th St. Oakland City Center
1,16th St. Mission
2,19th St. Oakland
3,24th St. Mission
4,Antioch


In [129]:
# Check if names are equal or one is a substring of the other 

def clean_station_name(new_crime_data):
    if "North Concord" in new_crime_data:
        return "North Concord/Martinez"
    elif '24th Street' in new_crime_data:
            return '24th St. Mission'
    elif "Antioch E-Bart" in new_crime_data:
            return "Antioch"
    elif "West Dublin" in new_crime_data:
        return "West Dublin/Pleasanton"
    elif "Civic Center" in new_crime_data:
        return "Civic Center/UN Plaza"
    elif '16th Street' in new_crime_data:
        return '16th St. Mission'
    elif"El Cerrito Del Nort" in new_crime_data:
        return "El Cerrito del Norte"
    elif "Berryessa/N San Jose" in new_crime_data:
        return "Berryessa/North San Jose"
    elif "SFO" in new_crime_data:
        return "San Francisco International Airport"
    elif "Powell" in new_crime_data:
        return "Powell St."
    elif "Pleasant Hill" in new_crime_data:
        return "Pleasant Hill/Contra Costa Centre"
    elif "12th Street" in new_crime_data:
        return "12th St. Oakland City Center"
    elif "Montgomery" in new_crime_data:
        return "Montgomery St."
    else:
        return new_crime_data

new_crime_data = new_crime_data.apply(clean_station_name)
new_crime_data

0                                Millbrae
1                             Balboa Park
2                             Embarcadero
3                                Milpitas
4                                Richmond
5                    San Francisco County
6                        24th St. Mission
7                                Coliseum
8                               Fruitvale
9                  North Concord/Martinez
10                                Antioch
11                              MacArthur
12                           West Oakland
13                 West Dublin/Pleasanton
14                       San Mateo County
15                           Walnut Creek
16                                  Ashby
17                  Civic Center/UN Plaza
18                             Union City
19                                Fremont
20                       16th St. Mission
21                   El Cerrito del Norte
22                           Lake Merritt
23                    South San Fr

In [130]:
# merge the crime data with the station data using outer join
merge_data = pd.merge(new_crime_data, new_station_data, left_on='location', right_on='name', how='outer')

# Display all rows of the merged data
pd.set_option('display.max_rows', None)
merge_data


,location,name
0,Millbrae,Millbrae
1,Balboa Park,Balboa Park
2,Embarcadero,Embarcadero
3,Milpitas,Milpitas
4,Richmond,Richmond
5,San Francisco County,NaN
6,24th St. Mission,24th St. Mission
7,Coliseum,Coliseum
8,Fruitvale,Fruitvale
9,North Concord/Martinez,North Concord/Martinez


In [131]:

# Create condition: if location matches name, then add it to a thrird column
merge_data['station_name'] = np.where(merge_data['location'] == merge_data['name'], merge_data['location'], merge_data['name'])

# Rename station_name column to crime_location
merge_data = merge_data.rename(columns={"station_name": "crime_location"})
# merge_data

# Create a new dataframe with only location and crime_location columns
view_crime_data = merge_data[['location', 'crime_location']]
view_crime_data

# Remove instances San Francisco County,  San Mateo County, Oakland Shops, Pittsburg, Oakland West, Berkeley
view_crime_data = view_crime_data[~view_crime_data['location'].isin(['San Francisco County', 'San Mateo County', 'Oakland Shops', 'Pittsburg', 'Oakland West', 'Berkeley', 'Not on Property'])]

# Drop location column
view_crime_data = view_crime_data.drop(columns='location')

# Drop duplicates
view_crime_data = view_crime_data.drop_duplicates()

# Export the data to a csv file
view_crime_data.to_csv('../Resources/view_crime_data.csv', index=False)
view_crime_data



,crime_location
0,Millbrae
1,Balboa Park
2,Embarcadero
3,Milpitas
4,Richmond
6,24th St. Mission
7,Coliseum
8,Fruitvale
9,North Concord/Martinez
10,Antioch
